In [1]:
import pandas as pd
import numpy as np
from openai import OpenAI
import pandas as pd
import numpy as np
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4")
from tqdm import tqdm
import pickle
from transformers import AutoModel, AutoTokenizer
from torch.nn.functional import cosine_similarity
import torch.nn.functional as F
import torch
import csv
import math



checkpoint="Salesforce/codet5p-110m-embedding"
tokenizer = AutoTokenizer.from_pretrained(checkpoint, trust_remote_code=True)
model = AutoModel.from_pretrained(checkpoint, trust_remote_code=True).to("cuda")
def get_code_embedding(code_snippet):
    """
    Generates embeddings for a given code snippet using a pre-trained model.

    Parameters:
    - code_snippet (str): The code for which embeddings are to be generated.
    - checkpoint (str): The model checkpoint to be used for embedding. Default is Salesforce/codet5p-110m-embedding.
    - device (str): Device to run the model on, either 'cuda' for GPU or 'cpu' for CPU. Default is 'cuda'.

    Returns:
    - torch.Tensor: Embedding tensor for the input code.
    """
    inputs = tokenizer.encode(code_snippet, return_tensors="pt").to("cuda")
    with torch.no_grad():
        embedding = model(inputs)[0]
    
    return embedding.cpu()

/u1/hfaheem/.conda/envs/.env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
file_path = "/u1/hfaheem/DLAndroidArtifact/my-paths-code/output3/results/android_services_methods.parquet"
df = pd.read_parquet(file_path)
df.head()


,service_name,class,method,depths,access control level,json_answer
0,Lcom.android.server.broadcastradio.BroadcastRa...,Lcom/android/server/broadcastradio/BroadcastRa...,addAnnouncementListener([ILandroid/hardware/ra...,"[{'depth': 0, 'java_code': 'depth : 0 public I...",SYS_OR_SIG,"{""Sinks"": [[""ICloseHandle v14 = v13.addAnnounc..."
1,Lcom.android.server.broadcastradio.BroadcastRa...,Lcom/android/server/broadcastradio/BroadcastRa...,listModules()Ljava/util/List;,"[{'depth': 0, 'java_code': 'depth : 0 public L...",SYS_OR_SIG,"{""Sinks"": [[""ArrayList v2 = new ArrayList();"",..."
2,Lcom.android.server.broadcastradio.BroadcastRa...,Lcom/android/server/broadcastradio/BroadcastRa...,openTuner(ILandroid/hardware/radio/RadioManage...,"[{'depth': 0, 'java_code': 'depth : 0 public I...",SYS_OR_SIG,"{""Sinks"": [[""ITuner v11 = v10.openSession(p1, ..."
3,Lcom.android.server.soundtrigger_middleware.So...,Lcom/android/server/soundtrigger_middleware/So...,<clinit>()V,"[{'depth': 0, 'java_code': 'depth : 0 public v...",NONE,"{""Sinks"": []}"
4,Lcom.android.server.soundtrigger_middleware.So...,Lcom/android/server/soundtrigger_middleware/So...,attachAsOriginator(ILandroid/media/permission/...,"[{'depth': 0, 'java_code': 'depth : 0 public I...",NONE,"{""Sinks"": [[""Object v1 = Objects.requireNonNul..."


In [49]:
import json 
def process_json_answer(json_answer, n=2): # todo: change n to float("inf")
    global counter
    all = []
    if isinstance(json_answer, str):  # If it's a string, parse it
        try:
            json_answer = json.loads(json_answer)
        except json.JSONDecodeError:
            # print("Invalid JSON format")
            return []
    try:
        arrays = json_answer['Sinks']
        for i, array in enumerate(arrays, 1):
            if i > n:  # Limit the number of joins to `n`
                break
            joined = '\n'.join(array)
            all.append(joined)
    except:    
        return []
    counter += 1
    return all

counter = 0
# Step 2: Apply to create a new column for joined json_answer
df['sink_code'] = df['json_answer'].apply(process_json_answer)

# display the first row first column of the DataFrame
print(f"total rows = {len(df)}")
print(f"row with valid json  = {counter}")
# df.head()
# df.iloc[0]["sink_code"]

total rows = 500
row with valid json  = 471


In [50]:
df["embeddings"] = None
for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing Rows"):
    code_snippets = row["sink_code"]
    embeddings = []
    for each in code_snippets:
        code_embedding = get_code_embedding(each)
        embeddings.append(code_embedding)
    df.at[index, "embeddings"] = embeddings


Processing Rows: 100%|██████████| 500/500 [00:06<00:00, 75.22it/s] 


Get the most similar code(prolly ignore)

In [42]:

similarities = {}


for index1, row1 in tqdm(df.iterrows(), total=len(df), desc="Computing Similarities"):
    ep1_id = row1["method"]  # name of the EP1
    ep1_embeddings = row1["embeddings"] # list of embeddings for the EP1
    ep1_sink_code = row1["sink_code"] # list of code snippets for the EP1
    
    closest_ep = None
    max_similarity = -1
    similar_sink_pair = None
    
    for index2, row2 in df.iterrows():
        if index1 == index2:
            continue  
        
        ep2_id = row2["method"] # name of the EP2
        ep2_embeddings = row2["embeddings"] # list of embeddings for the EP2
        ep2_sink_code = row2["sink_code"]  # list of code snippets for the EP2
        
        # Compute pairwise similarities between all embeddings of EP1 and EP2
        for i, emb1 in enumerate(ep1_embeddings):
            for j, emb2 in enumerate(ep2_embeddings):
                sim = torch.nn.functional.cosine_similarity(torch.tensor(emb1), torch.tensor(emb2), dim=0).item()
                if sim > max_similarity:
                    max_similarity = sim
                    closest_ep = ep2_id
                    similar_sink_pair = (ep1_sink_code[i], ep2_sink_code[j])  
    
    similarities[ep1_id] = {
        "closest_ep": closest_ep,
        "max_similarity": max_similarity,
        "similar_sink_pair": similar_sink_pair
    }
    


Computing Similarities:   0%|          | 0/500 [00:00<?, ?it/s]/tmp/ipykernel_6003/526392682.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sim = torch.nn.functional.cosine_similarity(torch.tensor(emb1), torch.tensor(emb2), dim=0).item()
Computing Similarities:   0%|          | 2/500 [00:00<00:46, 10.67it/s]

Computing Similarities: 100%|██████████| 500/500 [00:37<00:00, 13.49it/s]


In [ ]:
for ep, details in similarities.items():
    if details['similar_sink_pair'] is None or details['closest_ep'] is None:
        print(f"EP {ep} has no similar EPs.")
    else:
        print(f"EP {ep} is most similar to EP {details['closest_ep']} with similarity {details['max_similarity']:.4f}")
        print(f"Similar sink codes:\nEP {ep}: \n{details['similar_sink_pair'][0]}\nEP \n{details['closest_ep']}: \n{details['similar_sink_pair'][1]}")
    print("\n\n")

Get >0.8 similar

In [55]:
similarities = {}

for index1, row1 in tqdm(df.iterrows(), total=len(df), desc="Computing Similarities"):
    ep1_id = row1["method"] # name of the EP1
    ep1_embeddings = row1["embeddings"] # list of embeddings for the EP1
    ep1_sink_code = row1["sink_code"] # list of code snippets for the EP1
    
    similar_sink_pairs = []  # List to store all similar sink code pairs with similarity > 0.8
    
    for index2, row2 in df.iterrows():
        if index1 == index2:
            continue  
        
        ep2_id = row2["method"] # name of the EP2
        ep2_embeddings = row2["embeddings"] # list of embeddings for the EP2
        ep2_sink_code = row2["sink_code"] # list of code snippets for the EP2  
        
        # Compute similarities
        for i, emb1 in enumerate(ep1_embeddings):
            for j, emb2 in enumerate(ep2_embeddings):
                sim = torch.nn.functional.cosine_similarity(torch.tensor(emb1), torch.tensor(emb2), dim=0).item()
                if sim > 0.8:  # Only consider pairs with similarity > 0.5, we can filter later
                    similar_sink_pairs.append({
                        "similarity": sim,
                        "ep1_code": ep1_sink_code[i],
                        "ep2_id": ep2_id,
                        "ep2_code": ep2_sink_code[j]
                    })
    
    # Store all similar pairs for the current EP
    similarities[ep1_id] = similar_sink_pairs

Computing Similarities:   0%|          | 0/500 [00:00<?, ?it/s]/tmp/ipykernel_6003/2290205527.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sim = torch.nn.functional.cosine_similarity(torch.tensor(emb1), torch.tensor(emb2), dim=0).item()
Computing Similarities: 100%|██████████| 500/500 [00:37<00:00, 13.47it/s]


In [ ]:
# Display results
for ep, similar_pairs in similarities.items():
    if not similar_pairs:
        print(f"EP {ep} has no similar EPs with similarity > 0.8.")
    else:
        print(f"EP {ep} has the following similar sink code pairs with similarity > 0.8:")
        for pair in similar_pairs:
            print(f"  - Similarity: {pair['similarity']:.4f}")
            print(f"    EP {ep}: \n{pair['ep1_code']}")
            print(f"    EP {pair['ep2_id']}: \n{pair['ep2_code']}")
            print()
    print("\n\n")

Write >0.8 similar and unique with top similarity

In [9]:
CSV_FILE = './todelete'

# Write the data to a CSV file
with open(CSV_FILE+ "_allcode" + ".csv", mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)

    # Write header
    writer.writerow(["EP1_ID", "EP2_ID", "EP1_Code", "EP2_Code", "Similarity"])

    # Iterate through the data and write each entry
    for ep, similar_pairs in similarities.items():
        if similar_pairs:
            for pair in similar_pairs:
                writer.writerow([
                    ep,
                    pair['ep2_id'],
                    pair['ep1_code'].replace("\n", " "),  # Replace newlines for better CSV readability
                    pair['ep2_code'].replace("\n", " "),  # Replace newlines for better CSV readability
                    pair['similarity']
                ])
        else:
            writer.writerow([ep, "No similar EPs with similarity > 0.8", "", "", ""])

# Writing the data to a CSV file
with open(CSV_FILE + "_score" + ".csv", mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    # Write the header
    writer.writerow(["EP", "EP2_ID", "Max_Similarity"])
    
    for ep, similar_pairs in similarities.items():
        if not similar_pairs:
            writer.writerow([ep, "No similar EPs", ""])
        else:
            max_similarity_per_ep2 = {}
            for pair in similar_pairs:
                ep2_id = pair['ep2_id']
                similarity = pair['similarity']
                if ep2_id not in max_similarity_per_ep2:
                    max_similarity_per_ep2[ep2_id] = similarity
                else:
                    max_similarity_per_ep2[ep2_id] = max(max_similarity_per_ep2[ep2_id], similarity)
            
            # Sorting the EP2 IDs by similarity in descending order
            sorted_ep2_ids = sorted(max_similarity_per_ep2.items(), key=lambda x: x[1], reverse=True)
            
            # Writing sorted EP2 IDs and their max similarity
            for ep2_id, max_similarity in sorted_ep2_ids:
                writer.writerow([ep, ep2_id, f"{max_similarity:.4f}"])



with open(CSV_FILE + "_top2code" + ".csv", mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)

    # Write header
    writer.writerow(["EP1_ID", "EP2_ID", "EP1_Code", "EP2_Code", "Similarity"])

    # Iterate through the data and write only the top 2 most similar pairs
    for ep, similar_pairs in similarities.items():
        
        if similar_pairs:
            # Sort the pairs by similarity in descending order
            top_pairs = sorted(similar_pairs, key=lambda x: x["similarity"], reverse=True)[:2]
            for pair in top_pairs:
                writer.writerow([
                    ep,
                    pair['ep2_id'],
                    pair['ep1_code'].replace("\n", " "),  # Replace newlines for better CSV readability
                    pair['ep2_code'].replace("\n", " "),  # Replace newlines for better CSV readability
                    pair['similarity']
                ])
        else:
            writer.writerow([ep, "No similar EPs with similarity > 0.8", "", "", ""])

print(f"Data has been written to {CSV_FILE}")

Data has been written to ./todelete


In [52]:
def get_top_similar_methods(similarities, top_n=2, threshold=0.8):
    filtered = [entry for entry in similarities if entry['similarity'] > threshold]
    sorted_results = sorted(filtered, key=lambda x: x['similarity'], reverse=True)
    top_results = sorted_results[:top_n]
    extracted_results = [
        {
            'ep2_code': entry['ep2_code'],
            'ep2_id': entry['ep2_id'],
            'ep1_code': entry['ep1_code'],
            'similarity': entry['similarity']
        }
        for entry in top_results
    ]
    
    return extracted_results




In [45]:
df["json_answer"].value_counts()

json_answer
None                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    29
{"Sinks": []}                                                                                                                                                                                                                                                                                                                                                                    

In [59]:
counter = 0
no_top_similar_found = 0
df["json_answer2"] = None
df["access control level predicted"] = None


valid_methods = []
for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Processing rows"):
    if row["json_answer"] == "None" or row["json_answer"] == "{\"Sinks\": []}" or row["json_answer"] == "{\"Sinks\": [[]]}":
        continue
    counter += 1
    full_method_name = row['method']
    method_name = row['method'].split("(")[0]
    service_name = row['service_name']
    top_similar = get_top_similar_methods(similarities[full_method_name])
    
    if top_similar == []:
        no_top_similar_found += 1
        continue
        # print(service_name, method_name)
    valid_methods.append(full_method_name)
print(f"Total rows with valid JSON: {counter}")
print(f"Total rows with no top similar found: {no_top_similar_found}")

Processing rows: 100%|██████████| 500/500 [00:00<00:00, 9093.34it/s]

Total rows with valid JSON: 462
Total rows with no top similar found: 126


In [61]:
len(valid_methods)


# write in a textfile 

with open("valid_methods_to_delete.txt", "w") as f:
    for method in valid_methods:
        f.write(method + "\n")
        

In [ ]:
# read the text file in an array

with open("valid_methods_to_delete.txt", "r") as f:
    valid_methods = f.readlines()
    
valid_methods = [method.strip() for method in valid_methods]
len(valid_methods)